<a href="https://colab.research.google.com/github/Mainakdeb/moa-classification/blob/master/hyperparameter_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!cp /content/drive/"My Drive"/kaggle/lish-moa.zip /content/

In [ ]:
!unzip lish-moa.zip

In [ ]:
!pip install pip install iterative-stratification

In [ ]:
!pip install optuna

In [6]:
from sklearn.preprocessing import MinMaxScaler
        
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

import optuna
import numpy as np 
import pandas as pd 
from IPython.display import clear_output
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [7]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device


'cuda'

In [8]:
train_features = pd.read_csv('train_features.csv')
train_targets = pd.read_csv('train_targets_scored.csv')
train_targets_s = train_targets
test_features = pd.read_csv('test_features.csv')

ss = pd.read_csv('sample_submission.csv')

In [9]:
class TorchStandardScaler:
  def fit(self, x):
    self.mean = x.mean(0, keepdim=True)
    self.std = x.std(0, unbiased=False, keepdim=True)
  def transform(self, x):
    x -= self.mean
    x /= (self.std + 1e-7)
    return x

In [10]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

target = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [11]:
top_features = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]

all_columns = train.columns
train=train[all_columns[top_features]]
test = test[all_columns[top_features]]
train.shape, test.shape

((21948, 785), (3982, 785))

In [12]:
train = train.values
target = target.values
test = test.values

In [13]:
train.shape, target.shape

((21948, 785), (21948, 206))

In [14]:
class TrainDataset(Dataset):
    def __init__(self, train,targets, noise ):
        
        self.features  = train
        self.targets = targets
        self.noise = noise
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx]).float()
        target = torch.tensor(self.targets[idx]).float()
        return feature, target


In [15]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
def show_lr(learning_rates):
    plt.plot(learning_rates, label = "learning rate")
    plt.ylabel("Learning rate", fontsize = 15)
    plt.grid(True)
    plt.legend()
    plt.show()

def train_step(x, y, model, optimizer, criterion):
    optimizer.zero_grad()
    pred = model(x.to(device))
    y = y.float()
    loss = criterion(pred,y.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

In [16]:
class Model(nn.Module):
  def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout):
    super().__init__()
    layers = []
    for _ in range(nlayers):
      if len(layers) == 0:
        layers.append(nn.Linear(nfeatures, hidden_size))
        layers.append(nn.BatchNorm1d(hidden_size))
        layers.append(nn.Dropout(dropout))
        layers.append(nn.LeakyReLU())
      else:
        layers.append(nn.Linear(hidden_size, hidden_size))
        layers.append(nn.BatchNorm1d(hidden_size))
        layers.append(nn.Dropout(dropout))
        layers.append(nn.LeakyReLU())

    layers.append(nn.Linear(hidden_size, ntargets))

    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)

In [23]:
def run_training( params, save_model=False):
  NFOLDS = 5
  EPOCHS = 10 ## changes here 
  mskf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=0)

  fold_val_losses = list()

  for k , (train_idx,valid_idx) in enumerate(mskf.split(train,target)):

      x_train,x_valid,y_train,y_valid = train[train_idx,:],train[valid_idx,:],target[train_idx,:],target[valid_idx,:]
      
      train_dataset = TrainDataset(x_train, y_train, noise = False)
      valid_dataset = TrainDataset(x_valid, y_valid, noise = False)
      
      train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers = 8)
      val_loader = DataLoader(dataset=valid_dataset, batch_size=256, shuffle = True, num_workers = 8)
      

      model = Model(nfeatures=x_train.shape[1], 
                    ntargets=y_train.shape[1],
                    nlayers=params["num_layers"], 
                    hidden_size=params["hidden_size"], 
                    dropout=params["dropout"])
      
      model = model.cuda()
      optimizer = optim.Adam(model.parameters(), lr = params["learning_rate"], weight_decay=1e-5)
      scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                      mode='min', 
                                                      factor=0.5, 
                                                      patience=3, 
                                                      eps=1e-4, 
                                                      verbose=True)
      criterion = nn.BCEWithLogitsLoss()

      eng = Engine(model, optimizer, device='cuda')
      best_loss = 10000

      print(CRED ,"fold ", str(k+1), CEND)

      for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.evaluate(val_loader)
        print("train_loss:", train_loss, "val_loss:", valid_loss)
        if valid_loss<best_loss:
          best_loss = valid_loss
          if save_model:
            torch.save(model.state_dict(), "model_{fold}.pth")
      
  return(best_loss)

  print(CBLUE, "Training complete", CEND)

In [24]:
class Engine:
  def __init__(self, model, optimizer, device):
    self.model = model
    self.device = device
    self.optimizer = optimizer

  @staticmethod
  def loss_fn(targets, outputs):
    return nn.BCEWithLogitsLoss()(outputs, targets)

  def train(self, data_loader):
    self.model.train()
    final_loss=0
    for data in data_loader:
      self.optimizer.zero_grad()
      inputs, targets = data
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = self.model(inputs)
      loss = self.loss_fn(targets, outputs)
      loss.backward()
      self.optimizer.step()
      final_loss += loss.item()
    return(final_loss / len(data_loader))


  def evaluate(self, data_loader):
    self.model.train()
    final_loss=0
    for data in data_loader:
      #self.optimizer.zero_grad()
      inputs, targets = data
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = self.model(inputs)
      loss = self.loss_fn(targets, outputs)
      #loss.backward()
      #self.optimizer.step()
      final_loss += loss.item()
    return(final_loss / len(data_loader))

In [28]:
# params = {
#       "num_layers":trial.suggest_int("num_layer", 1, 8),
#       "hidden_size":trial.suggest_int("hidden_size", 16, 4096),
#       "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
#       "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2)
#   }
# run_training(params, save_model=False)

In [26]:
def objective(trial):
  params = {
      "num_layers":trial.suggest_int("num_layer", 1, 8),
      "hidden_size":trial.suggest_int("hidden_size", 16, 4096),
      "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
      "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2)
  }

  loss_ = run_training(params, save_model=False)
  return(loss_)

In [27]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

print("best_trial:")
trial_ = study.best_trial
print(trial_)

[I 2020-09-26 05:54:06,147] A new study created in memory with name: no-name-ee38c3fd-5b1d-436c-aaf4-9469cbda9cf5


 fold  1 
train_loss: 0.043743044785831284 val_loss: 0.021343196638756327
train_loss: 0.02116268497986206 val_loss: 0.020676683220598433
train_loss: 0.020207170693986656 val_loss: 0.019720713194045756
train_loss: 0.01940260318013421 val_loss: 0.018940692353579733
train_loss: 0.01882745702818468 val_loss: 0.018534616981115606
train_loss: 0.018510718789437542 val_loss: 0.01847604838096433
train_loss: 0.01823858735219076 val_loss: 0.01821031028197871
train_loss: 0.01812999578786717 val_loss: 0.01797429658472538
train_loss: 0.017969645674515894 val_loss: 0.01779964416184359
train_loss: 0.017864463236722826 val_loss: 0.01830590350760354
 fold  2 
train_loss: 0.04391568210785803 val_loss: 0.021582691412833
train_loss: 0.021215533716199192 val_loss: 0.020386476380129654
train_loss: 0.020119138035005417 val_loss: 0.019981973701053195
train_loss: 0.019343324747962364 val_loss: 0.01923203778763612
train_loss: 0.018833740344406037 val_loss: 0.01918898171020879
train_loss: 0.01846751036203426 val_

[I 2020-09-26 05:56:41,592] Trial 0 finished with value: 0.01809049728843901 and parameters: {'num_layer': 6, 'hidden_size': 1052, 'dropout': 0.5188347969850853, 'learning_rate': 0.0008831212239556775}. Best is trial 0 with value: 0.01809049728843901.


 fold  1 
train_loss: 0.03650541653505702 val_loss: 0.020441576424572203
train_loss: 0.019983187884740208 val_loss: 0.019850569466749828
train_loss: 0.019189611880405657 val_loss: 0.01866108277398679
train_loss: 0.01876079143308427 val_loss: 0.018565644199649494
train_loss: 0.018553689000723156 val_loss: 0.01841845146069924
train_loss: 0.018383346437274115 val_loss: 0.01832727508412467
train_loss: 0.01838922692273838 val_loss: 0.01821949312256442
train_loss: 0.018229413237692654 val_loss: 0.018506406289007928
train_loss: 0.018156288657337427 val_loss: 0.018654074002471235
train_loss: 0.0180750146670186 val_loss: 0.018389592775040202
 fold  2 
train_loss: 0.03636835280643857 val_loss: 0.0203969848031799
train_loss: 0.01993950075753357 val_loss: 0.01944553293287754
train_loss: 0.019059326202757118 val_loss: 0.01889761785666148
train_loss: 0.018667442406919123 val_loss: 0.018903950850168865
train_loss: 0.018492234526607004 val_loss: 0.018950306706958346
train_loss: 0.01843927412604292 val

[I 2020-09-26 05:59:25,976] Trial 1 finished with value: 0.018089856228066817 and parameters: {'num_layer': 7, 'hidden_size': 721, 'dropout': 0.3384970726057638, 'learning_rate': 0.0019213365993020527}. Best is trial 1 with value: 0.018089856228066817.


...

 fold  1 
train_loss: 0.04480706496785084 val_loss: 0.01965240202844143
train_loss: 0.018951686267889498 val_loss: 0.018580824240214296
train_loss: 0.017851890361719372 val_loss: 0.01774555134276549
train_loss: 0.01723722595235576 val_loss: 0.017622331985168986
train_loss: 0.016857454155072355 val_loss: 0.017353243608441617
train_loss: 0.016451478348639997 val_loss: 0.017123380882872477
train_loss: 0.01602077712237403 val_loss: 0.017353584420763783
train_loss: 0.015656640908370417 val_loss: 0.01718384638014767
train_loss: 0.015278107790357393 val_loss: 0.017141662538051605
train_loss: 0.014787716075670029 val_loss: 0.017187018775277667
 fold  2 
train_loss: 0.04436535760760307 val_loss: 0.019786341529753473
train_loss: 0.01886998097398791 val_loss: 0.018504344547788303
train_loss: 0.01784546678021982 val_loss: 0.01807844576736291
train_loss: 0.017158294278804376 val_loss: 0.018108613478640716
train_loss: 0.016781424313945616 val_loss: 0.017725515593257215
train_loss: 0.0162953116485606

[I 2020-09-26 06:43:14,867] Trial 18 finished with value: 0.017234986130562093 and parameters: {'num_layer': 5, 'hidden_size': 1787, 'dropout': 0.316428024102369, 'learning_rate': 0.00043538971847463726}. Best is trial 9 with value: 0.016764250118285418.


 fold  1 
train_loss: 0.02978992919721033 val_loss: 0.02009709044877026
train_loss: 0.019737743887294462 val_loss: 0.019398805271420214
train_loss: 0.018983170687072518 val_loss: 0.01851717082576619
train_loss: 0.01863463698764858 val_loss: 0.01818245856298341
train_loss: 0.018496409706447437 val_loss: 0.018281709868460894
train_loss: 0.01837314075479905 val_loss: 0.018558475499351818
train_loss: 0.018412175365602194 val_loss: 0.017958281096071005
train_loss: 0.018441458941315828 val_loss: 0.018352523342602782
train_loss: 0.01846834774925441 val_loss: 0.01837344912605153
train_loss: 0.01839180171246762 val_loss: 0.018632179747025173
 fold  2 
train_loss: 0.029388575517720936 val_loss: 0.020360642733673256
train_loss: 0.01947236443073421 val_loss: 0.01900698451532258
train_loss: 0.018816998464635748 val_loss: 0.01906106931467851
train_loss: 0.018716178024592606 val_loss: 0.018713584169745445
train_loss: 0.018366377152826473 val_loss: 0.01864211613105403
train_loss: 0.01831304626809298 v

[I 2020-09-26 06:45:28,621] Trial 19 finished with value: 0.018270036826531093 and parameters: {'num_layer': 3, 'hidden_size': 1419, 'dropout': 0.4523398838430158, 'learning_rate': 0.00401867937543168}. Best is trial 9 with value: 0.016764250118285418.


best_trial:
FrozenTrial(number=9, value=0.016764250118285418, datetime_start=datetime.datetime(2020, 9, 26, 6, 20, 33, 402214), datetime_complete=datetime.datetime(2020, 9, 26, 6, 22, 47, 319401), params={'num_layer': 3, 'hidden_size': 1894, 'dropout': 0.15739205763019304, 'learning_rate': 0.0011353568729844193}, distributions={'num_layer': IntUniformDistribution(high=8, low=1, step=1), 'hidden_size': IntUniformDistribution(high=4096, low=16, step=1), 'dropout': UniformDistribution(high=0.7, low=0.1), 'learning_rate': LogUniformDistribution(high=0.01, low=1e-06)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=9, state=TrialState.COMPLETE)
